In [1]:
import sqlite3
import json
from datetime import datetime



In [2]:
######################
#TUTORIAL code-block 1
######################

timeframe = '2012-05'
#build one transaction and connect data all at once
sql_transaction = []

connection = sqlite3.connect('{}.db'.format(timeframe))
#cursor
c = connection.cursor()

def create_table() :
    c.execute("""CREATE TABLE IF NOT EXISTS parent_reply
    (parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, 
    parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT)""")

def format_data(data):
    data = data.replace("\n", " newlinechar ").replace("\r", " newlinechar ").replace('"', "'")
    return data

def acceptable(data):
    if len(data.split(' ')) > 50 or len(data) < 1:
        return False
    elif len(data) > 1000:
        return False
    elif data == '[deleted]' or data == '[removed]':
        return False
    else:
        return True
    


def find_parent(pid):
    try:
        sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0] # 0 cause we are only selecting 1 commenting
        else:
            return False
    except Exception as e:
        #print("Find Parent)
        return False
    

In [3]:
def find_existing_score(pid):
    try:
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0] # 0 cause we are only selecting 1 commenting
        else:
            return False
    except Exception as e:
        #print("Find Parent)
        return False 

In [4]:
def transaction_bldr(sql):
    #globalling the sql_trnsaction variable
    global sql_transaction
    #keep building it till it's a certain size
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
    connection.commit()
    sql_transaction = []


def sql_insert_replace_comment(commentid, parentid, parent, comment, subreddit, time, score):
    try:
        sql = """UPDATE parent_reply SET parent_id = ?, comment_id = ?,
        parent = ?, comment = ?, subreddit = ?, unix = ?, score = ?
        WHERE parent_id = ?;""".format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion', str(e))
        
    
def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment,
        subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},
        {});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))


def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit,
        unix, score) VALUES ("{}","{}","{}","{}",{},
        {});""".format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))   

In [ ]:
#parent_data = find_parent(parent_id)    
if __name__ == "__main__":
        create_table()
        #how many rows we have created
        row_counter = 0
        #how many parents and child pairs we found
        paired_rows = 0
        
        with open('RC_2009-04', buffering = 1000) as f:
            for row in f:
                row_counter += 1
                row = json.loads(row)
                parent_id = row['parent_id']
                body = format_data(row['body'])
                comment_id = row['name']
                created_utc = row['created_utc']
                score = row['score']
                subreddit = row['subreddit']
                parent_data = find_parent(parent_id)
                
                
                
                #since there is a lot of useless comments on reddit,
                #we only keep certain comments
                if score >= 2: #means somebody saw this comment at least 
                    if acceptable(body):
                        #now we check to see if there's a comment with this parent id with a better score
                        existing_comment_score = find_existing_score(parent_id)
                        if existing_comment_score:
                            if score > existing_comment_score:
                                ####
                                sql_insert_replace_comment(comment_id, parent_id, parent_data, body, subreddit, created_utc, score)
                        else:
                            if parent_data:
                                sql_insert_has_parent(comment_id, parent_id, parent_data, body, subreddit, created_utc, score)
                            else:
                                sql_insert_no_parent(comment_id, parent_id, body, subreddit, created_utc, score)
                            
                    
                    

('s0 insertion', "'ascii' codec can't encode characters in position 0-1: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2014' in position 77: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xac' in position 38: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2013' in position 172: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2299' in position 0: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xe0' in position 40: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode characters in position 28-29: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2022' in position 0: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xa9' in position 131: ordinal not in range(128)")
('s0 insertion', "'ascii' codec 

('s0 insertion', "'ascii' codec can't encode character u'\\u2026' in position 25: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u04a8' in position 0: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xa0' in position 4: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2019' in position 129: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2019' in position 103: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2019' in position 6: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xa3' in position 88: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u04a8' in position 0: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u0c20' in position 200: ordinal not in range(128)")
('s0 insertion'

('s0 insertion', "'ascii' codec can't encode character u'\\u2026' in position 55: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode characters in position 21-23: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xe9' in position 147: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xe9' in position 5: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u201c' in position 4: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u1edf' in position 3: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xf6' in position 86: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u0142' in position 3: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xc9' in position 11: ordinal not in range(128)")
('s0 insertion', "'ascii' co

('s0 insertion', "'ascii' codec can't encode character u'\\u4eba' in position 33: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2014' in position 18: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2019' in position 8: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xe4' in position 68: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u201c' in position 4: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode characters in position 56-62: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2019' in position 274: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2014' in position 212: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u5927' in position 4: ordinal not in range(128)")
('s0 insertion', "'a

('s0 insertion', "'ascii' codec can't encode character u'\\u2014' in position 134: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u201c' in position 151: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode characters in position 4-9: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode characters in position 0-5: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode characters in position 0-3: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xe4' in position 137: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2665' in position 2: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2019' in position 88: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2019' in position 7: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't 

...

('s0 insertion', "'ascii' codec can't encode characters in position 0-1: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode characters in position 0-1: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode characters in position 0-3: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode characters in position 0-14: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u201c' in position 0: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u0ca0' in position 0: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2014' in position 41: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xf3' in position 35: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u0ca0' in position 177: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode cha

('s0 insertion', "'ascii' codec can't encode character u'\\xe9' in position 5: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xe4' in position 1: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xe9' in position 7: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u203d' in position 58: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xa0' in position 43: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2019' in position 6: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2026' in position 59: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2019' in position 38: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xbf' in position 0: ordinal not in range(128)")
('s0 insertion', "'ascii'

('s0 insertion', "'ascii' codec can't encode character u'\\u2026' in position 153: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2014' in position 121: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2019' in position 18: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xe9' in position 129: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2261' in position 38: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xe1' in position 230: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xb4' in position 4: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2014' in position 74: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xa3' in position 37: ordinal not in range(128)")
('s0 insertion'

('s0 insertion', "'ascii' codec can't encode characters in position 0-2: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u25d4' in position 0: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2665' in position 214: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xe9' in position 13: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xe9' in position 5: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2014' in position 9: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xe1' in position 29: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u201c' in position 17: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2026' in position 8: ordinal not in range(128)")
('s0 insertion', "'ascii' co

('s0 insertion', "'ascii' codec can't encode character u'\\ufb41' in position 3: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u0ca0' in position 0: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u201c' in position 0: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xe0' in position 36: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2019' in position 110: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xbf' in position 5: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\xbf' in position 0: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2260' in position 16: ordinal not in range(128)")
('s0 insertion', "'ascii' codec can't encode character u'\\u2018' in position 213: ordinal not in range(128)")
('s0 insertion', "'

In [ ]:
######################
#TUTORIAL code-block 1
######################


